###Import essential libraries

In [2]:
from PIL import Image
import os
import re

import matplotlib.pyplot as plt
import numpy as np

import pylab as pl
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from skimage.feature import hog

Obtain image feature & label

In [3]:
folder='a1_TrainingPhotos/' # location of training images
images = []
i=0
X=[]
Y=[]

for filename in os.listdir(folder):   # run the loop for all files in training folder
    img = Image.open(os.path.join(folder,filename)) # open image 
    img=img.resize([32,64])  # resize image  to a fixed size
    fd, hog_image = hog(img, orientations=9, pixels_per_cell=(4,4), cells_per_block=(2, 2), visualize=True, channel_axis=-1) # obtain HOG features
    
    X.append(fd)
    # obtain training labels from file name
    if re.match("Elon*", filename):
      Yt=0
    elif re.match("Jeff*", filename):
      Yt=1
    elif re.match("Bill*", filename):
      Yt=2
    elif re.match("Mark*", filename):
      Yt=3
    elif re.match("Steve*", filename):
      Yt=4
    else:
      Yt=5
    Y.append(Yt)
    i=i+1


In [4]:
fd.shape

(3780,)

In [5]:
target_names=['Elon_Musk', 'Jeff_Bezos', 'Bill_Gates', 'Mark_Zuckerberg', 'Steve_Jobs']

Split data into training/test

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Reduce dimenions with PCA

In [7]:
n_components = .95
pca = PCA(n_components=n_components, whiten=True).fit(X_train)

In [8]:
# project training and test data to lower dimension 
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [9]:
# Save PCA coefficients for use at prediction

import pickle as pk
pk.dump(pca, open("c1_FR_PCA.pkl","wb"))

Train SVM classifier

In [10]:
print("Fitting the classifier to the training set")
param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5],
          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
          }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced',probability=True), param_grid)
clf = clf.fit(X_train_pca, y_train)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Fitting the classifier to the training set
Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.001, probability=True)


In [11]:
# Save best SVM classifier weights for use at prediction

import joblib
joblib.dump(clf.best_estimator_, 'c2_FR_svm_classifier.pkl', compress = 1) # Only best parameters

['c2_FR_svm_classifier.pkl']

Model performance on test set

In [12]:
y_pred = clf.predict(X_test_pca)
y_pred

array([1, 4, 0, 2, 2, 3, 0, 3, 0, 1, 4, 1, 0, 1, 4, 3, 3, 0, 3, 0, 4, 1,
       2, 2, 0, 3, 3, 2, 1, 0, 4, 1, 2, 1, 0, 0, 0, 2, 0, 4, 0, 0, 4, 0,
       1, 2, 2, 3, 0, 2, 4, 4, 3, 4, 0, 2, 0, 3, 0, 0, 2, 0, 4, 1, 2, 3,
       4, 2, 2, 2, 1, 2, 3, 4, 0, 1, 1, 3, 4, 4, 4, 4, 1, 2, 0, 1, 0, 0,
       0, 3, 4, 3, 0, 0, 2, 1, 2, 2, 2, 3, 1, 1, 4, 1, 1, 4, 3, 0, 3, 0,
       1, 2, 2, 2, 0, 4, 4, 1, 1, 0, 1, 0, 2, 4, 0, 4, 1, 3, 0, 0, 4, 3,
       3, 0, 1, 1, 1, 3, 4, 3, 1, 4, 4, 1, 0, 1, 4, 3, 4, 1, 2, 2, 4, 2,
       2, 4, 2, 3, 3, 4, 0, 1, 3, 0, 0, 4, 3, 2, 1, 3, 4, 1, 0, 4, 2, 1,
       1, 4, 1, 1, 3, 1, 1, 0, 0, 4, 0, 0, 3, 4, 2, 3, 3, 3, 3, 1, 4, 4,
       1, 1, 3, 1, 0, 3])

In [14]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87        45
           1       0.96      0.93      0.95        46
           2       0.94      0.91      0.93        35
           3       0.81      0.88      0.85        34
           4       0.95      0.89      0.92        44

    accuracy                           0.90       204
   macro avg       0.90      0.90      0.90       204
weighted avg       0.90      0.90      0.90       204



Coding ends here! Switch back to predictor notebook.